In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
driver = webdriver.Edge()
wait = WebDriverWait(driver, 10)

In [ ]:
base_url = "https://hesetazegi.ir/fa/food-culture/loc-isfahan?page={}"
page = 1

while True:
    print(f"\n--- Visiting page {page} ---")
    driver.get(base_url.format(page))
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.group")))

    group_links = driver.find_elements(By.CSS_SELECTOR, "a.group")
    group_urls = [link.get_attribute('href') for link in group_links if link.get_attribute('href')]

    for url in group_urls:
        print(f"\nVisiting group URL: {url}")
        driver.get(url)
        try:
            # Wait for the container to load
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.w-full.h-fit.rounded-xl.border.border-border-secondary')))
        except:
            print("No ingredient container found.")
            continue

        # Find all ingredient rows
        ingredient_rows = driver.find_elements(By.CSS_SELECTOR, 
            'div.w-full.h-fit.rounded-xl.border.border-border-secondary '
            'div.flex.justify-between.p-3.w-full.h-fit.items-center'
        )

        for row in ingredient_rows:
            try:
                # Extract name
                name_elem = row.find_element(By.CSS_SELECTOR, '.flex.gap-3.items-center .flex-col p')
                ingredient_name = name_elem.text.strip()

                # Extract amount + unit
                amount_elem = row.find_element(By.CSS_SELECTOR, 'p.text-body-md.text-label-secondary')
                amount_text = amount_elem.text.strip()

                print(f"- {ingredient_name}: {amount_text}")
            except Exception as e:
                print(f"Skipped one row due to error: {e}")

        time.sleep(1)
        driver.back()
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.group")))

    # Go to next pagination page if exists
    try:
        next_page_selector = f'nav[aria-label="Pagination"] a[href*="page={page + 1}"]'
        driver.find_element(By.CSS_SELECTOR, next_page_selector)
        page += 1
    except:
        print("No more pages.")
        break

driver.quit()
